# Housing prices

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from IPython.display import display
pd.options.display.max_columns = None

## Data loading

In [3]:
train = pd.read_csv("train.csv", index_col=0)
X_test = pd.read_csv("test.csv", index_col=0)

In [4]:
X = train.iloc[:, :-1]
y = train.iloc[:, -1:]

## Data preprocessing
### Missing values

In [5]:
cols_with_missing = X.isnull().any()
cols_with_missing = cols_with_missing[cols_with_missing].index

cols_missing_factor = pd.Series(X[cols_with_missing].isnull().sum() / len(X) * 100,
                                index=cols_with_missing)

MISSING_VALUES_THRESHOLD = 50

In [6]:
values = ((col, round(perc, 1), perc >= MISSING_VALUES_THRESHOLD) for col, perc in cols_missing_factor.iteritems())
columns = ["Columns with missing values", "percentage of missing values", "to drop"]

pd.DataFrame(values, columns=columns)

,Columns with missing values,percentage of missing values,to drop
0,LotFrontage,17.7,False
1,Alley,93.8,True
2,MasVnrType,0.5,False
3,MasVnrArea,0.5,False
4,BsmtQual,2.5,False
5,BsmtCond,2.5,False
6,BsmtExposure,2.6,False
7,BsmtFinType1,2.5,False
8,BsmtFinType2,2.6,False
9,Electrical,0.1,False


In [7]:
cols_to_drop = cols_missing_factor[cols_missing_factor >= MISSING_VALUES_THRESHOLD].index.to_list()
X = X.drop(columns=cols_to_drop)
X_test = X_test.drop(columns=cols_to_drop)

In [8]:
X.columns = range(X.shape[1])
X_test.columns = range(X.shape[1])

num_cols = X.select_dtypes(include=np.number).columns.to_list()
cat_cols = X.select_dtypes(exclude=np.number).columns.to_list()

In [9]:
from sklearn.impute import SimpleImputer

num_imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
X[num_cols] = num_imputer.fit_transform(X[num_cols])
X_test[num_cols] = num_imputer.transform(X_test[num_cols])

cat_imputer = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])
X_test[cat_cols] = cat_imputer.transform(X_test[cat_cols])

In [10]:
X = X.values
y = y.values
X_test = X_test.values

### Splitting to train and validation set


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=0)

### Features standarization

In [12]:
from sklearn.preprocessing import StandardScaler

X_std_scaler = StandardScaler()
X_train[:, num_cols] = X_std_scaler.fit_transform(X_train[:, num_cols])
X_val[:, num_cols] = X_std_scaler.transform(X_val[:, num_cols])

y_std_scaler = StandardScaler()
y_train = y_std_scaler.fit_transform(y_train)
y_val = y_std_scaler.transform(y_val)

### Encoding categorical features

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

col_trans = ColumnTransformer([("encoder", OneHotEncoder(), cat_cols)], remainder="passthrough")
col_trans.fit(X)
X_train = col_trans.transform(X_train).toarray()
X_val = col_trans.transform(X_val).toarray()

### Preview

In [14]:
pd.DataFrame(X_train)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274
0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.868365,0.978678,0.103212,2.131506,-0.513041,1.188032,1.078914,1.913588,-0.901062,-0.291823,2.757198,1.865729,1.787570,-0.802555,-0.125918,0.601886,-0.820747,-0.236888,0.792769,-0.773624,0.146285,-0.203898,1.512184,0.605438,1.189777,1.683633,1.434743,-0.742575,0.874116,-0.364703,-0.115333,4.546911,-0.058085,-0.092588,0.256396,-0.613562
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.868365,-0.467156,-0.372881,-0.794852,-0.513041,-0.292501,-1.097548,-0.568183,-1.011427,-0.291823,0.745229,-0.387262,-0.715410,-0.802555,-0.125918,-1.216718,-0.820747,-0.236888,-1.037179,-0.773624,-1.100873,-0.203898,-0.951178,-0.955603,-0.666587,-1.057111,-0.791166,-0.742575,-0.700461,-0.364703,-0.115333,-0.269109,-0.058085,-0.092588,0.623394,0.884118
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

## Linear regression

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

lin_reg = LinearRegression().fit(X_train, y_train)

y_pred = lin_reg.predict(X_val)
lin_reg_score = mean_absolute_error(y_val, y_pred)
print("MAE:", lin_reg_score)

MAE: 5223538662.163562


## Polynomial regression

In [16]:
from sklearn.preprocessing import PolynomialFeatures

X_train_poly = X_train[:, num_cols]
X_train_poly = PolynomialFeatures(degree=2).fit_transform(X_train)
X_train_poly = np.concatenate((X_train_poly, np.delete(X_train_poly, num_cols, axis=1)), axis=1)

X_val_poly = X_val[:, num_cols]
X_val_poly = PolynomialFeatures(degree=2).fit_transform(X_val)
X_val_poly = np.concatenate((X_val_poly, np.delete(X_val_poly, num_cols, axis=1)), axis=1)

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

poly_reg = LinearRegression().fit(X_train_poly, y_train)

y_pred = poly_reg.predict(X_val_poly)
poly_reg_score = mean_absolute_error(y_val, y_pred)
print("MAE:", poly_reg_score)

MAE: 0.2605722816039472


## Support vector regression

In [18]:
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error

svr = SVR(kernel="rbf").fit(X_train, y_train)

y_pred = svr.predict(X_val)
svr_score = mean_absolute_error(y_val, y_pred)
print("MAE:", svr_score)

MAE: 0.2356523856123345


## Decision tree regression

In [19]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

tree = DecisionTreeRegressor().fit(X_train, y_train)

y_pred = tree.predict(X_val)
tree_score = mean_absolute_error(y_val, y_pred)
print("MAE:", tree_score)

MAE: 0.35539613785409396


## Random forest regression

In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

forest = RandomForestRegressor().fit(X_train, y_train)

y_pred = forest.predict(X_val)
forest_score = mean_absolute_error(y_val, y_pred)
print("MAE:", forest_score)

MAE: 0.22259153970703194


## Gradient boosting

In [21]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

xgb = XGBRegressor().fit(X_train, y_train)

y_pred = xgb.predict(X_val)
xgb_score = mean_absolute_error(y_val, y_pred)
print("MAE:", xgb_score)

MAE: 0.2230903366219941


## Neural network 

In [23]:
from keras import models, layers
from sklearn.metrics import mean_absolute_error

net = models.Sequential()
net.add(layers.Dense(128, input_shape=(X_train.shape[1],), activation="relu"))
net.add(layers.Dropout(0.2))
net.add(layers.Dense(64, activation="relu"))
net.add(layers.Dense(1, activation=None))

net.compile(optimizer="rmsprop", loss="mae", metrics=["mae"])

history = net.fit(X_train, y_train, batch_size=32, validation_data=(X_val, y_val), epochs=100, verbose=0)

y_pred = net.predict(X_val)
nn_score = mean_absolute_error(y_val, y_pred)
print("MAE:", nn_score)

MAE: 0.21622128839812574
